In [3]:
pip install qiskit-aer

  Obtaining dependency information for qiskit-aer from https://files.pythonhosted.org/packages/a0/ef/1d2d9e18d8a735876fe862408248e850cd264769eebb307c7de52f10abdc/qiskit_aer-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached qiskit_aer-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.0 kB)
Using cached qiskit_aer-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/a7/68/16e7b9154fae61fb29f0f3450b39b855b89e6d2c598d67302e70f96883af/matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/67/0f/6e5b4879594cd1cbb6a2754d9230937be444f404cf07c360c07a10b36aac/contourpy-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached contourpy-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Obtaining depen

In [18]:
import numpy as np
from qiskit_aer import Aer
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector

# Definizione dei coefficienti
alpha = np.sqrt(0.3)
beta = np.sqrt(0.7)

# Creare un circuito quantistico con 3 qubit e 2 bit classici
qc = QuantumCircuit(3, 2)

# Preparare lo stato |ψ⟩ = α|0⟩ + β|1⟩ sul primo qubit
norm = np.sqrt(alpha**2 + beta**2)
qc.initialize([alpha/norm, beta/norm], 0)

# Creare una coppia di Bell tra il secondo e il terzo qubit
qc.h(1)
qc.cx(1, 2)


# Alice esegue una misura di Bell sul primo e secondo qubit
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])

# Applicare le operazioni correttive a seconda delle misure di Alice
qc.x(2).c_if(qc.clbits[0], 1)  # Se il primo bit misurato è 1, applicare X
qc.z(2).c_if(qc.clbits[1], 1)  # Se il secondo bit misurato è 1, applicare Z

backend = Aer.get_backend('statevector_simulator')
job = backend.run(qc)
result = job.result()
statevector = result.get_statevector()


In [19]:
# Convertire il vettore di stato in un oggetto Statevector
statevector = Statevector(statevector)

# Estrarre le probabilità degli stati del terzo qubit
prob_zero = statevector.probabilities([2])[0]  # probabilità di qubit 2 essere 0
prob_one = statevector.probabilities([2])[1]   # probabilità di qubit 2 essere 1

# Ricostruire lo stato finale di Bob
state_bob = np.array([np.sqrt(prob_zero), np.sqrt(prob_one)])

# Stato iniziale di Alice
state_alice = np.array([alpha/norm, beta/norm])

# Confrontare lo stato di Bob con lo stato iniziale di Alice
success = np.allclose(state_bob, state_alice, atol=1e-2)
print("Statevector finale di Bob:", state_bob)
print("Stato iniziale di Alice:", state_alice)
print("Teletrasporto riuscito:", success)

Statevector finale di Bob: [0.54772256 0.83666003]
Stato iniziale di Alice: [0.54772256 0.83666003]
Teletrasporto riuscito: True
